In [16]:
import pandas as pd
from pathlib import Path
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder

## Inspect Dataframe

def inspect_dataframe(df)

In [17]:
def inspect_dataframe(df):
    # ทำสำเนาเพื่อความปลอดภัย (กันการแก้ไขของเดิมโดยไม่ตั้งใจ)
    df_safe = df.copy()

    print(f"Loaded {len(df_safe):,} rows × {len(df_safe.columns):,} columns")
    display(df_safe.head())

    print("\nDataFrame info:")
    df_safe.info()

    print("\nMissing values per column:")
    print(df_safe.isna().sum())
    print("----------------------------------------")

## Remove row that province not in bangkok

filter_bangkok(df)

In [18]:
def filter_bangkok(df):
    """ลบแถวที่ province ไม่ใช่กรุงเทพฯ / Bangkok และแสดง before–after แบบ 1 บรรทัด"""
    print(f"before: {len(df)}", end="  ->  ")
    df = df[df['province'].astype(str).str.contains(r'กรุงเทพ|Bangkok', case=False, na=False)]
    print(f"after: {len(df)}")
    print("----------------------------------------")
    return df

## drop_rows_with_nan

drop_rows_with_nan(df, cols, inplace=False)

In [19]:
def drop_rows_with_nan(df, cols, inplace=False):
    """
    Drop rows that have NaN in any column listed in `cols`.
    - df: pandas DataFrame
    - cols: list-like of column names to check for NaN
    - inplace: if True, modify df in-place and return it; otherwise return a new DataFrame
    """
    cols = list(cols)
    missing_cols = [c for c in cols if c not in df.columns]
    if missing_cols:
        raise ValueError(f"Columns not found in DataFrame: {missing_cols}")

    before = len(df)
    result = df.dropna(subset=cols)
    after = len(result)
    print(f"Dropped {before - after} rows with NaN in {cols} (before: {before}, after: {after})")
    print("----------------------------------------")

    if inplace:
        # replace contents of original DataFrame
        df.drop(df.index, inplace=True)
        for col in result.columns:
            df[col] = result[col]
        return df
    return result

## Drop Columns

drop_columns(df, cols, inplace=False, ignore_missing=False)

In [20]:
def drop_columns(df, cols, inplace=False, ignore_missing=False):
    """
    Drop columns by name.
    - df: pandas DataFrame
    - cols: list-like of column names to drop
    - inplace: if True, modify df in-place and return it; otherwise return a new DataFrame
    - ignore_missing: if True, silently ignore names not present in df; otherwise raise KeyError
    """
    cols = list(cols)
    missing = [c for c in cols if c not in df.columns]
    if missing:
        if ignore_missing:
            cols = [c for c in cols if c in df.columns]
        else:
            raise KeyError(f"Columns not found: {missing}")

    if inplace:
        df.drop(columns=cols, inplace=True)
        print(f"Dropped columns (in-place): {cols}")
        return df
    result = df.drop(columns=cols)
    print(f"Dropped columns (returned new df): {cols}")
    print("----------------------------------------")
    return result

## One hot encoder

one_hot_encode(df, cols, method='sklearn', drop_first=False, inplace=False, return_encoder=False)

In [21]:
def one_hot_encode_sklearn(df, cols, drop_first=False, inplace=False, return_encoder=False, verbose=True):
    """
    One-hot encode `cols` using sklearn.OneHotEncoder (compatible with old/new sklearn).
    Logs actions if verbose=True.
    - df: pandas DataFrame
    - cols: list-like of column names to encode
    - drop_first: if True drop first level (use drop='first' in OneHotEncoder)
    - inplace: if True, replace contents of df and return it; otherwise return a new DataFrame
    - return_encoder: if True return (result_df, encoder)
    - verbose: if True print logs
    """
    cols = list(cols)
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Columns not found: {missing}")

    if verbose:
        print(f"One-hot encoding columns: {cols}")
        print(f"drop_first={drop_first}, inplace={inplace}, return_encoder={return_encoder}")

    enc_kwargs = {"drop": "first" if drop_first else None, "handle_unknown": "ignore"}
    # instantiate encoder with compatibility for different sklearn versions
    try:
        enc = OneHotEncoder(sparse_output=False, **enc_kwargs)
        api = "sparse_output"
    except TypeError:
        enc = OneHotEncoder(sparse=False, **enc_kwargs)
        api = "sparse"

    if verbose:
        print(f"Using OneHotEncoder API variant: {api}")

    arr = enc.fit_transform(df[cols])
    new_cols = list(enc.get_feature_names_out(cols))
    if verbose:
        print(f"Generated {len(new_cols)} columns, first 10: {new_cols[:10]}")

    df_ohe = pd.DataFrame(arr, index=df.index, columns=new_cols)
    before_shape = df.shape
    result = pd.concat([df.drop(columns=cols), df_ohe], axis=1)
    after_shape = result.shape

    if verbose:
        print(f"Shape before: {before_shape} -> after: {after_shape}")
        print("----------------------------------------")

    if inplace:
        # replace contents of original DataFrame while keeping same object
        df.drop(df.index, inplace=True)
        for c in result.columns:
            df[c] = result[c]
        if verbose:
            print("Replaced original DataFrame contents (in-place).")
        if return_encoder:
            return df, enc
        return df

    if return_encoder:
        return result, enc
    return result

## Fill NaN



In [22]:
def fill_missing_values(df, cols, fills, inplace=False, verbose=True):
    """Fill NaN for cols using corresponding fills (cols and fills must match length)."""
    if len(cols) != len(fills): raise ValueError("cols and fills must have same length")
    res = df if inplace else df.copy()
    for c, f in zip(cols, fills):
        if c not in res.columns:
            if verbose: print(f"⚠ '{c}' not found — skip")
            print("----------------------------------------")
            continue
        if isinstance(f, dict) and not f or f is None:
            val = "Not define"
        else:
            val = f
        before = res[c].isna().sum()
        res[c] = res[c].fillna(val)
        if verbose:
            filled = before - int(res[c].isna().sum())
            print(f"✔ {c}: filled {filled} with {repr(val)}")
            print("----------------------------------------")
    return res

## Parse date time

safe_parse_datetime(df, col, fmt)

In [23]:
def safe_parse_datetime(df, col, fmt):
    df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce')
    before = len(df)
    df = df.dropna(subset=[col])
    after = len(df)
    print(f"{col}: before {before} -> after {after}")
    print("----------------------------------------")
    return df

## Eliminate Outliner

In [24]:
def eliminate_outliers(df, cols="all_numeric", method="iqr_cap",
                       factor=1.5, inplace=False, print_info=True):
    """
    Remove or cap outliers using the IQR method.

    Parameters:
    - df: pandas DataFrame
    - cols: list-like of column names, or "all_numeric" to apply on all numeric columns
    - method: "iqr_cap" = truncate values, "iqr_remove" = remove rows
    - factor: IQR multiplier (default 1.5)
    - inplace: modify df in-place if True
    - print_info: print summary for each column

    Returns:
    - DataFrame with outliers handled
    """

    if not inplace:
        df = df.copy()

    # Automatically select numeric columns
    if cols == "all_numeric":
        cols = df.select_dtypes(include="number").columns.tolist()

    else:
        cols = list(cols)

    # Check existence
    for col in cols:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame.")

    # Process each column
    for col in cols:
        if not pd.api.types.is_numeric_dtype(df[col]):
            if print_info:
                print(f"Skipping non-numeric column: {col}")
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower = Q1 - factor * IQR
        upper = Q3 + factor * IQR

        if method == "iqr_cap":
            df[col] = df[col].clip(lower, upper)

        elif method == "iqr_remove":
            df = df[(df[col] >= lower) & (df[col] <= upper)]

        else:
            raise ValueError("method must be 'iqr_cap' or 'iqr_remove'")

        if print_info:
            print(f"[{col}]")
            print(f"  Q1 = {Q1:.4f}, Q3 = {Q3:.4f}, IQR = {IQR:.4f}")
            print(f"  lower = {lower:.4f}, upper = {upper:.4f}")
            print(f"  method = {method}")
            print("----------------------------------------------------")

    return df


## call Function

In [25]:
# download data
path = Path("./bangkok_traffy.csv")
if not path.exists():
    raise FileNotFoundError(f"{path} not found in {Path.cwd()}")

df_raw = pd.read_csv(path)

In [26]:
df = df_raw.copy()  # work on a copy to keep raw data intact

# 0.) inspect
inspect_dataframe(df)

# 1.) remove duplicates ID by keep last
df2 = df.drop_duplicates(subset='ticket_id', keep='last')
print(f"Removed {len(df) - len(df2)} duplicate ticket_id rows (kept last).")
print("----------------------------------------------------")

# 2.) filter bangkok in province
df = filter_bangkok(df)

# 3.) drop rows with nan
df = drop_rows_with_nan(df, ["district", "subdistrict"])

# 4.) drop columns that not use
print("drop_columns size before", df.shape)
df = drop_columns(df, ["photo","photo_after","ticket_id"])
print("drop_columns size after", df.shape)
print("----------------------------------------------------")

# 5.1) One hot
df = one_hot_encode_sklearn(df, ['state','star'])
df = drop_columns(df, ["star_nan"])
#? df = one_hot_encode_sklearn(df, ["organization"])

# ?5.2) One hot of type
# df['type'] = (
#     df['type']
#     .str.replace(r'[\{\}]', '', regex=True)
#     .str.split(',')
# )

# 6.) fill missing value with "text"
fill_missing_values(df, 
                    ["comment","organization","type"], 
                    ["ไม่มีคำอธิบาย","ไม่ระบุหน่วยงาน","ไม่ระบุ"])

# 7.) split coords to lon and lat
df[['lon','lat']] = df['coords'].str.split(',', expand=True).astype(float)

# 8.) Convert timestamp และ last_activity เป็น datetime
fmt = "%Y-%m-%d %H:%M:%S.%f%z"
df = safe_parse_datetime(df, 'timestamp', fmt)
df = safe_parse_datetime(df, 'last_activity', fmt)

# 9.) new Column Solve date
df['solve_days'] = (df['last_activity'] - df['timestamp']).dt.days

Loaded 787,026 rows × 16 columns


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787026 entries, 0 to 787025
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ticket_id      778254 non-null  object 
 1   type           786929 non-null  object 
 2   organization   786455 non-null  object 
 3   comment        778254 non-null  object 
 4   photo          786911 non-null  object 
 5   photo_after    641309 non-null  object 
 6   coords         787026 non-null  object 
 7   address        778254 non-null  object 
 8   subdistrict    786460 non-null  object 
 9   district       786465 non-null  object 
 10  province       786831 non-null  object 
 11  timestamp      787026 non-null  object 
 12  state          787026 non-null  object 
 13  star           274097 non-null  float64
 14  count_reopen   787026 non-null  int64  
 15  last_activity  787026 non-null  object 
dtypes: float64(1), int64(1), object(14)
memory usage: 96.1+ M

In [27]:
# 10.1) eliminate outliner : solve_days
col = "solve_days"

mean_val = df[col].mean()
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower = 0
upper = Q3 + 2 * IQR
print(f"[{col}] mean={mean_val:.4f}, Q1={Q1:.4f}, Q3={Q3:.4f}, IQR={IQR:.4f}, lower={lower:.4f}, upper={upper:.4f}")
df_asc = df.sort_values(by=col, ascending=True)
print("=== ASCENDING SORT ===")
print(df_asc[[col]].head(300))   # log 300 rows แรก
df_desc = df.sort_values(by=col, ascending=False)
print("=== DESCENDING SORT ===")
print(df_desc[[col]].head(20))  # log 20 rows แรก
print("count of non-NaN values:", df[col].count())
print(f"number of col over {upper:.4f}: ", (df[col] > upper).sum())

print("Before eliminating outliers", df[col].count())
df = df[(df[col] >= lower) & (df[col] <= upper)]
print("After eliminating outliers", df[col].count())

[solve_days] mean=73.9108, Q1=1.0000, Q3=71.0000, IQR=70.0000, lower=0.0000, upper=211.0000
=== ASCENDING SORT ===
        solve_days
295920          -1
367246          -1
410774          -1
528415          -1
534478          -1
...            ...
786684           0
340              0
786715           0
786714           0
786713           0

[300 rows x 1 columns]
=== DESCENDING SORT ===
       solve_days
7            1073
61           1046
29           1021
52           1008
96           1006
77           1004
67            992
110           970
166           962
380           961
136           956
3260          956
832           955
879           955
1545          954
1694          953
8624          952
8623          952
12415         952
8622          952
count of non-NaN values: 785610
number of col over 211.0000:  99586
Before eliminating outliers 785610
After eliminating outliers 685891


In [28]:
# 10.2) eliminate outliner : count_reopen
col = "count_reopen"

mean_val = df[col].mean()
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower = 0
upper = 2
print(f"[{col}] mean={mean_val:.4f}, Q1={Q1:.4f}, Q3={Q3:.4f}, IQR={IQR:.4f}, lower={lower:.4f}, upper={upper:.4f}")
print(f"count {col} not a 0 & na : ", (df[col].ne(0) & df[col].notna()).sum(), f"mean not a 0 & na : ", (df.loc[df[col].ne(0) & df[col].notna(), col].mean()))

df_asc = df.sort_values(by=col, ascending=True)
print("=== ASCENDING SORT ===")
print(df_asc[[col]].head(300))   # log 300 rows แรก
df_desc = df.sort_values(by=col, ascending=False)
print("=== DESCENDING SORT ===")
print(df_desc[[col]].head(20))  # log 20 rows แรก
print("count of non-NaN values:", df[col].count())
print(f"number of col over {upper:.4f}: ", (df[col] > upper).sum())

print("Before eliminating outliers", df[col].count())
print(lower, upper)
df = df[(df[col] >= lower) & (df[col] <= upper)]
print("After eliminating outliers", df[col].count())

[count_reopen] mean=0.1128, Q1=0.0000, Q3=0.0000, IQR=0.0000, lower=0.0000, upper=2.0000
count count_reopen not a 0 & na :  43696 mean not a 0 & na :  1.7699331746612963
=== ASCENDING SORT ===
     count_reopen
49              0
55              0
56              0
70              0
80              0
..            ...
646             0
647             0
648             0
649             0
650             0

[300 rows x 1 columns]
=== DESCENDING SORT ===
        count_reopen
670683            67
624874            65
304095            65
340128            61
247441            59
660043            58
694861            58
659926            57
315620            53
653762            47
346084            45
327611            45
295728            44
272219            43
326695            43
167540            42
287463            41
671731            41
704230            39
345165            39
count of non-NaN values: 685891
number of col over 2.0000:  6120
Before eliminating outliers 685891
0 

In [29]:
df.shape
# before : Loaded 787,026 rows × 16 columns
# Truncate Outliers after : Loaded 785,610 rows × 23 columns
# Remove Outliners after : Loaded 392,070 rows × 23 columns

(679771, 22)

In [30]:
# load .csv
df.to_csv("clean_data.csv", index=False, encoding="utf-8-sig")